# Introduction

Presidential approval polls are a key metric for understanding public sentiment of political events. However, traditional polling can be time and labor intensive. With the rise of social media, we have at our fingertips a wealth of data on public sentiment of any number of topics, including politics. The goal of this project is to build a model that uses social media comments to generate a presidential approval rating.

## Approval Rating Data

Presidential approval ratings have been collected from [FiveThirtyEight](https://projects.fivethirtyeight.com/biden-approval-rating/). For this project, I will be using only data from Biden's presidency, so the dates in this dataset are January 20, 2021 - September 15, 2021.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from pmaw import PushshiftAPI
import datetime as dt

from textblob import TextBlob

In [2]:
biden = pd.read_csv('Data/Biden.csv')

In [3]:
biden.head()

,president,subgroup,modeldate,startdate,enddate,pollster,grade,samplesize,population,weight,...,disapprove,adjusted_approve,adjusted_disapprove,multiversions,tracking,url,poll_id,question_id,createddate,timestamp
0,Joseph R. Biden Jr.,All polls,9/15/2021,1/21/2021,2/2/2021,Gallup,B+,906.0,a,1.314707,...,37.0,56.394227,36.431592,NaN,NaN,https://news.gallup.com/poll/329348/biden-begi...,74344,139651,2/4/2021,9/15/2021 13:24
1,Joseph R. Biden Jr.,All polls,9/15/2021,1/28/2021,2/1/2021,Quinnipiac University,A-,1075.0,a,1.532490,...,36.0,51.375610,35.903037,NaN,NaN,https://poll.qu.edu/poll-release?releaseid=3766,74334,139617,2/3/2021,9/15/2021 13:24
2,Joseph R. Biden Jr.,All polls,9/15/2021,1/27/2021,2/1/2021,Global Strategy Group/GBAO (Navigator Research),B/C,1005.0,rv,0.915957,...,39.0,52.497920,38.182669,NaN,NaN,https://navigatorresearch.org/wp-content/uploa...,74324,139564,2/2/2021,9/15/2021 13:24
3,Joseph R. Biden Jr.,All polls,9/15/2021,1/28/2021,2/1/2021,AP-NORC,NaN,1055.0,a,1.427064,...,38.0,57.398180,38.411874,NaN,NaN,https://apnorc.org/wp-content/uploads/2021/02/...,74336,139629,2/4/2021,9/15/2021 13:24
4,Joseph R. Biden Jr.,All polls,9/15/2021,1/28/2021,2/1/2021,Rasmussen Reports/Pulse Opinion Research,B,1500.0,lv,0.565827,...,45.0,53.402310,39.062956,NaN,T,https://www.rasmussenreports.com/public_conten...,74325,139568,2/2/2021,9/15/2021 13:24


In [4]:
biden.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1595 entries, 0 to 1594
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   president            1595 non-null   object 
 1   subgroup             1595 non-null   object 
 2   modeldate            1595 non-null   object 
 3   startdate            1595 non-null   object 
 4   enddate              1595 non-null   object 
 5   pollster             1595 non-null   object 
 6   grade                1541 non-null   object 
 7   samplesize           1587 non-null   float64
 8   population           1595 non-null   object 
 9   weight               1595 non-null   float64
 10  influence            1595 non-null   float64
 11  approve              1595 non-null   float64
 12  disapprove           1595 non-null   float64
 13  adjusted_approve     1595 non-null   float64
 14  adjusted_disapprove  1595 non-null   float64
 15  multiversions        24 non-null     o

In [5]:
biden.drop(columns=['subgroup', 'president', 'modeldate', 'multiversions', 'tracking', 'url', 
                 'poll_id', 'question_id', 'createddate', 'timestamp'], inplace=True)

In [6]:
biden.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1595 entries, 0 to 1594
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   startdate            1595 non-null   object 
 1   enddate              1595 non-null   object 
 2   pollster             1595 non-null   object 
 3   grade                1541 non-null   object 
 4   samplesize           1587 non-null   float64
 5   population           1595 non-null   object 
 6   weight               1595 non-null   float64
 7   influence            1595 non-null   float64
 8   approve              1595 non-null   float64
 9   disapprove           1595 non-null   float64
 10  adjusted_approve     1595 non-null   float64
 11  adjusted_disapprove  1595 non-null   float64
dtypes: float64(7), object(5)
memory usage: 149.7+ KB


## Collecting Social Media Data

For this project, I will use data from Reddit. Because reddit is structured into various subreddits by interest, it is easy both to filter for political content and ensure better representation of a variety of viewpoints. I will collect data from six different subreddits: r/politics and r/news (the two largest general political subreddits), r/liberal and r/democrats (two of the largest left-leaning political subreddits) and r/conservative and r/libertarian (two of the largest right-leaning political subreddits).

In [7]:
api = PushshiftAPI()

before = int(dt.datetime(2021,9,15,0,0).timestamp())
after = int(dt.datetime(2021,1,20,0,0).timestamp())

subs = ['politics', 'news', 'conservative', 'liberal', 'libertarian', 'democrats']

In [8]:
def get_data(subreddit):
    limit = 100000
    comments = api.search_comments(subreddit=subreddit, limit=limit, before=before, after=after)
    df = pd.DataFrame(comments)
    return df

def combine_data(subs):
    full_df = pd.DataFrame()
    for sub in subs:
        new_df = get_data(sub)
        full_df = pd.concat([full_df, new_df])
    return full_df

In [9]:
reddit = combine_data(subs)

Total:: Success Rate: 84.67% - Requests: 1194 - Batches: 120 - Items Remaining: 0
Total:: Success Rate: 90.25% - Requests: 1118 - Batches: 112 - Items Remaining: 0
Total:: Success Rate: 85.17% - Requests: 1187 - Batches: 119 - Items Remaining: 0
69913 result(s) not found in Pushshift
Total:: Success Rate: 77.23% - Requests: 470 - Batches: 47 - Items Remaining: 1
1 result(s) not found in Pushshift
Total:: Success Rate: 88.17% - Requests: 1150 - Batches: 115 - Items Remaining: 0
Total:: Success Rate: 93.84% - Requests: 1201 - Batches: 121 - Items Remaining: 0


In [12]:
reddit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 530086 entries, 0 to 99999
Data columns (total 50 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   all_awardings                    530086 non-null  object 
 1   associated_award                 0 non-null       object 
 2   author                           530086 non-null  object 
 3   author_flair_background_color    138693 non-null  object 
 4   author_flair_css_class           20339 non-null   object 
 5   author_flair_template_id         46511 non-null   object 
 6   author_flair_text                76278 non-null   object 
 7   author_flair_text_color          156693 non-null  object 
 8   awarders                         509693 non-null  object 
 9   body                             530086 non-null  object 
 10  collapsed_because_crowd_control  0 non-null       object 
 11  comment_type                     0 non-null       object 
 12  cre

In [13]:
reddit.drop(columns = ['all_awardings', 'associated_award', 'author_flair_background_color', 
                        'author_flair_css_class', 'author_flair_template_id', 'author_flair_text', 
                        'author_flair_text_color', 'awarders', 'collapsed_because_crowd_control', 
                        'comment_type', 'gildings', 'id', 'is_submitter', 'link_id', 'locked', 'no_follow', 'parent_id', 
                        'retrieved_on', 'send_replies', 'stickied', 'subreddit_id', 'top_awarded_type', 'treatment_tags',
                        'author_flair_richtext', 'author_flair_type', 'author_fullname', 'author_patreon_flair', 
                        'author_premium', 'distinguished', 'author_cakeday', 'collapsed_reason_code', 
                        'archived', 'body_sha1', 'can_gild', 'collapsed', 'collapsed_reason', 'controversiality', 
                        'gilded', 'retrieved_utc', 'score_hidden', 'subreddit_name_prefixed', 'subreddit_type', 
                        'edited'], inplace=True)

reddit['created_utc'] = pd.to_datetime(reddit['created_utc'], unit='s')

In [14]:
reddit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 530086 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   author                 530086 non-null  object        
 1   body                   530086 non-null  object        
 2   created_utc            530086 non-null  datetime64[ns]
 3   permalink              530086 non-null  object        
 4   score                  530086 non-null  int64         
 5   subreddit              530086 non-null  object        
 6   total_awards_received  530086 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 32.4+ MB


## Sentiment Analysis

Finally, I will use TextBlob to generate sentiment scores for the data. 

In [15]:
reddit['body'] = reddit['body'].astype(str)

def subjectivity(text):
    return TextBlob(text).sentiment.subjectivity

def polarity(text):
    return TextBlob(text).sentiment.polarity

reddit['Polarity'] = reddit['body'].apply(polarity)
reddit['Subjectivity'] = reddit['body'].apply(subjectivity)

In [16]:
def get_sentiment(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else: return 'Positive'
    
reddit['Sentiment'] = reddit['Polarity'].apply(get_sentiment)

In [17]:
reddit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 530086 entries, 0 to 99999
Data columns (total 10 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   author                 530086 non-null  object        
 1   body                   530086 non-null  object        
 2   created_utc            530086 non-null  datetime64[ns]
 3   permalink              530086 non-null  object        
 4   score                  530086 non-null  int64         
 5   subreddit              530086 non-null  object        
 6   total_awards_received  530086 non-null  int64         
 7   Polarity               530086 non-null  float64       
 8   Subjectivity           530086 non-null  float64       
 9   Sentiment              530086 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(5)
memory usage: 44.5+ MB


In [18]:
reddit.head()

,author,body,created_utc,permalink,score,subreddit,total_awards_received,Polarity,Subjectivity,Sentiment
0,[deleted],[removed],2021-04-01 17:17:59,/r/politics/comments/mhmik7/ap_poll_finds_bide...,1,politics,0,0.000000,0.000000,Neutral
1,Flip2428,Growing up there I was always dumbfounded on h...,2021-04-01 17:17:58,/r/politics/comments/mhy1zm/new_mexico_is_set_...,1,politics,0,-0.051389,0.368056,Negative
2,boatpile,"Gaetz didn't even say ""no age you can't be sex...",2021-04-01 17:17:58,/r/politics/comments/mhuv7u/theres_no_age_that...,1,politics,0,0.412500,0.820833,Positive
3,theombudsmen,"I'm sure everyone knows this, but to clarify t...",2021-04-01 17:17:56,/r/politics/comments/mhzm4r/the_gop_rightly_fe...,1,politics,0,0.100000,0.853472,Positive
4,Tony_Chu,Hang up decorations with zip ties and cheer th...,2021-04-01 17:17:54,/r/politics/comments/mhulza/biden_must_clean_u...,1,politics,0,0.000000,0.000000,Neutral


In [19]:
reddit.to_csv('Data/reddit_data.csv')
biden.to_csv('Data/approval_data.csv')